## Load media

In [6]:
import pandas as pd

marine_medium = pd.read_csv("cgem/media/minimal_marine_medium_cgem.tsv", sep="\t", header=None)
marine_medium.columns = ["reaction", "flux"]
marine_medium_dict = marine_medium.set_index("reaction").squeeze().to_dict()
cpd_classification = pd.read_csv("cgem/media/media_compound_classification.tsv", sep="\t", header=None).set_index(0).squeeze().to_dict()

## Import individual models

In [2]:
import cobra

gems = {
    "synechococcus": cobra.io.read_sbml_model('../published_gems/synechococcus/synechococcus_standard.xml'),
    "prochlorococcus": cobra.io.read_sbml_model('../published_gems/prochlorococcus/prochlorococcus_standard.xml'),
    "pelagibacter": cobra.io.read_sbml_model("../published_gems/pelagibacter/pelagibacter_standard.xml"),
    "polaribacter": cobra.io.read_sbml_model("../published_gems/polaribacter/polaribacter_standard.xml"),
    "roseobacter": cobra.io.read_sbml_model("../published_gems/roseobacter/roseobacter_standard.xml"),
}

## Prepare abundances for regions and years

In [5]:
import pandas as pd 

revabund2015 = pd.read_csv("data/gifford/relative_abundance_2015.tsv", sep="\t")
revabund2016 = pd.read_csv("data/gifford/relative_abundance_2016.tsv", sep="\t")

ecoregions = {
    'West': [3, 4, 5, 6, 7],
    'North': [9, 11, 12],
    'Central': [16, 18, 20],
    'East': [22, 24, 26]
}

abundances = {}
for year in ["2015", "2016"]:
    abundances[year] = {}
    if year == "2015":
        revabund = revabund2015
    else:
        revabund = revabund2016
    for column in revabund.columns[2:]:
        station_id = column.split('_')[1]
        station_dict = dict(zip(revabund['model_id'], revabund[column]))
        abundances[year][station_id] = station_dict

revabund2015.head()

,Family,model_id,station_1_2015,station_2_2015,station_3_2015,station_4_2015,station_5_2015,station_6_2015,station_7_2015,station_9_2015,station_10_2015,station_11_2015,station_12_2015,station_14_2015,station_16_2015,station_18_2015,station_20_2015,station_22_2015,station_24_2015,station_26_2015
0,Pelagibacteraceae,pelagibacter,0.359867,0.518295,0.358698,0.478018,0.261016,0.580756,0.381236,0.447231,0.378441,0.529429,0.469869,0.296386,0.471836,0.388728,0.452238,0.419691,0.428781,0.482228
1,Flavobacteriaceae,polaribacter,0.157932,0.101184,0.308335,0.244592,0.374868,0.193729,0.316161,0.220303,0.229351,0.224854,0.219727,0.178257,0.256721,0.167921,0.189217,0.154038,0.228683,0.166933
2,Synechococcaceae,synechococcus,0.349722,0.284376,0.204091,0.145848,0.255324,0.134235,0.217462,0.155939,0.248483,0.081926,0.142410,0.397814,0.113415,0.299942,0.164355,0.195100,0.157830,0.161941
3,Rhodobacteraceae,roseobacter,0.068360,0.079809,0.104025,0.105373,0.079486,0.065077,0.056941,0.100095,0.090994,0.086076,0.086128,0.091710,0.116391,0.077784,0.084009,0.079855,0.095773,0.076877
4,Prochloraceae,prochlorococcus,0.064119,0.016336,0.024852,0.026169,0.029306,0.026203,0.028200,0.076432,0.052730,0.077715,0.081865,0.035834,0.041638,0.065624,0.110181,0.151316,0.088932,0.112021


## Prepare MICOM taxa file

In [16]:
from src.helpers import write_taxa_file

gems = {
    "synechococcus": 'published_gems/synechococcus/synechococcus_standard.xml',
    "prochlorococcus": 'published_gems/prochlorococcus/prochlorococcus_standard.xml',
    "pelagibacter": "published_gems/pelagibacter/pelagibacter_standard.xml",
    "polaribacter": "published_gems/polaribacter/polaribacter_standard.xml",
    "roseobacter": "published_gems/roseobacter/roseobacter_standard.xml",
}

model_ids, model_paths = list(gems.keys()), list(gems.values())

taxa_dfs = []
for year, year_abund in abundances.items():

    for ecoregion, stations in ecoregions.items():
        eco_abundances = {
            station: year_abund[str(station)]
            for station in stations
            if str(station) in year_abund
            }

        for station, relavabund in eco_abundances.items():
            taxa = write_taxa_file(
                f"{year}_{ecoregion}_{station}",
                model_ids=model_ids,
                abundances=[relavabund[model_id] for model_id in model_ids],
                model_paths=model_paths,
                )
            taxa_dfs.append(taxa)

taxa_df = pd.concat(taxa_dfs)
taxa_df.reset_index(drop=True, inplace=True)
taxa_df.to_csv("../cgem/outputs/cgem_taxa_by_station.tsv", sep="\t", index=False)

## Build cGEM

In [17]:
%%capture
from micom.workflows import build

manifest = build(
    taxonomy=taxa_df,
    model_db=None,
    out_folder="../cgem/communities",
    cutoff=0.0001,
    threads=14 ,
    solver="hybrid"
    )

## Add possitive growth constraint to each cGEM and fix photon bounds

In [1]:
import os
from micom import load_pickle

min_growth = 1e-4
model_dir = "../cgem/communities"
growth_rxn_ids = [
    "BOF__synechococcus",
    "Growth__pelagibacter",
    "Growth__polaribacter",
    "Growth__roseobacter",
    "BIOMASS__prochlorococcus"
]

model_paths = [
    os.path.join(model_dir, f)
    for f in os.listdir(model_dir)
    if f.endswith(".pickle")
    ]

for model_path in model_paths:
    cgem = load_pickle(model_path)

    for rxn in growth_rxn_ids:
        cgem.reactions.get_by_id(rxn).lower_bound = min_growth

    cgem.reactions.get_by_id("EX_photon_e__prochlorococcus").upper_bound = 0
    cgem.reactions.get_by_id("EX_photon_e__synechococcus").upper_bound = 0
    cgem.reactions.get_by_id("EX_photon_e__prochlorococcus").lower_bound = -1000
    cgem.reactions.get_by_id("EX_photon_e__synechococcus").lower_bound = -1000
    cgem.reactions.get_by_id("EX_photon_m").upper_bound = 0

    cgem.to_pickle(model_path)

In [16]:
from micom import load_pickle

cgem = load_pickle("../cgem/communities/2016_West_3.pickle")
cgem

Name,2016_West_3
Memory address,7f1d100c9dc0
Number of metabolites,5862
Number of reactions,7823
Number of genes,3489
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__synechococcus, e__synechococcus, p__synechococcus, m__synechococcus, u__synechococcus, x__synechococcus, a__synechococcus, m, c__prochlorococcus, car__prochlorococcus, e__prochlorococcus, p__prochlorococcus, th__prochlorococcus, p__pelagibacter, c__pelagibacter, e__pelagibacter, p__polaribacter, c__polaribacter, e__polaribacter, p__roseobacter, c__roseobacter, e__roseobacter"


In [3]:
cgem.abundances

id
synechococcus      0.025014
prochlorococcus    0.025014
pelagibacter       0.484935
polaribacter       0.299318
roseobacter        0.165719
Name: abundance, dtype: float64

## Assign marine medium

In [7]:
from src.helpers import assign_medium

cgem = assign_medium(cgem, marine_medium_dict)

sol_coop = cgem.cooperative_tradeoff(fraction=0.5, fluxes=True)
sol_coop

,abundance,growth_rate,reactions,metabolites
compartments,,,,
medium,NaN,NaN,379,379
pelagibacter,0.484935,0.209122,1525,1133
polaribacter,0.299318,0.929812,1932,1313
prochlorococcus,0.025014,0.077705,992,800
roseobacter,0.165719,0.514796,2165,1473
synechococcus,0.025014,0.075626,830,764
